In [1]:
'''
requirements for demo
'''
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from snowflake.snowpark.functions import udf
import datetime as dt
import numpy as np
import seaborn as sns
import dill
import random

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler


from config import snowflake_conn_prop
from snowflake.snowpark import version

sour_table_name='XURAN_TABLE'
dest_table_name='TABLE_ORIG'

def initialization():
    session = Session.builder.configs(snowflake_conn_prop).create()
    session.sql('use schema Team1_xuran;').collect()
    session.sql('drop table if exists Xuran_table;').collect()
    session.sql('create table Xuran_table as select * from "UWM_HOUSEHOLDING"."HOUSEHOLDING"."GAGE_TEST";').collect()
    session.sql('drop table if exists table_orig;').collect()
    session.sql('create table table_orig as select * from xuran_table;').collect()
    session.sql('alter table table_orig rename column FIRST_NAME to P1_FIRSTNAME;').collect()
    session.sql('alter table table_orig rename column MIDDLE_NAME to P1_MIDDLENAME;').collect()
    session.sql('alter table table_orig rename column LAST_NAME to P1_LASTNAME;').collect()
    session.sql('alter table table_orig rename column ADDRESS_LINE_1 to P1_ADDRESS_LINE_1;').collect()
    session.sql('alter table table_orig rename column ADDRESS_LINE_2 to P1_ADDRESS_LINE_2;').collect()
    session.sql('alter table table_orig rename column ADDRESS_LINE_3 to P1_ADDRESS_LINE_3;').collect()
    session.sql('alter table table_orig rename column CITY to P1_CITY;').collect()
    session.sql('alter table table_orig rename column STATE to P1_STATE;').collect()
    session.sql('alter table table_orig rename column ZIP to P1_ZIP;').collect()
    session.sql('alter table table_orig rename column CONTACT_ID to P1_CONTACT_ID;').collect()
    session.sql('alter table table_orig drop column FULL_NAME;').collect()
    session.sql('alter table table_orig drop column HOUSEHOLD_ID;').collect()
    session.sql('alter table table_orig drop column HOUSEHOLDED_IND;').collect()
    

def encode(s: str):
        if "None" in s:
            s = s.replace("None", " ")
        elif re.search(r'Ave[.]*$', s):
            s = s.replace("Ave", "Avenue")
        elif re.search(r'Av[.]*$', s):
            s = s.replace("Av", "Avenue")
        elif re.search(r'St[.]*$', s):
            s = s.replace("St", "Street")
        elif re.search(r'Rd[.]*$', s):
            s = s.replace("Rd", "Road")
        elif re.search(r'Dr[.]*$', s):
            s = s.replace('Dr', "Drive")
        return s

'''
register a new user with given info
use per-trained model
'''
def register(firstName,lastName,addressLine1,addressLine2,addressLine3,city,state,zipCode):
    session = Session.builder.configs(snowflake_conn_prop).create()
    print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
    raw = session.table(dest_table_name)
    data = raw.toPandas().drop(columns=['P1_CONTACT_ID','P1_FIRSTNAME', 'P1_MIDDLENAME',
                       'P1_LASTNAME','P1_ADDRESS_LINE_2',], axis=5)
    data['P2_ADDRESS_LINE_1']=[addressLine1 for i in range(len(data))]
    data['P2_ADDRESS_LINE_3']=[addressLine3 for i in range(len(data))]
    data['P2_CITY']=[city for i in range(len(data))]
    data['P2_STATE']=[state for i in range(len(data))]
    data['P2_ZIP']=[zipCode for i in range(len(data))]
    data["P2_ADDRESS_LINE_1"]= data.apply(lambda row : encode(str(row["P2_ADDRESS_LINE_1"])), axis = 1).map(str)
    data["P2_ADDRESS_LINE_3"]= data.apply(lambda row : encode(str(row["P2_ADDRESS_LINE_3"])), axis = 1).map(str)
    with open('model.pkl', 'rb') as in_strm:
        model = dill.load(in_strm)
    
    pred = model.predict_proba(data)[:,1]
    predictions = [round(value) for value in pred]
    if 0 not in predictions:
        noFamilyAssigned(firstName,lastName,addressLine1,addressLine2,addressLine3,city,state,zipCode)
    else:
        familyAssigned(firstName,lastName,addressLine1,addressLine2,addressLine3,city,state,zipCode,predictions.index(0))
    return 0
    
    
def familyAssigned(firstName,lastName,addressLine1,addressLine2,addressLine3,city,state,zipCode,index):
    session = Session.builder.configs(snowflake_conn_prop).create()
    raw = session.table(sour_table_name)
    data= raw.toPandas()
    
    contact_id=str(len(data)+1)
    full_name=firstName+' '+lastName
    session.sql('INSERT INTO '+ sour_table_name+' VALUES ('+ 
            contact_id+',\''+full_name+'\',\''+firstName+'\',\'\',\''+lastName+'\',\''+
            addressLine1+'\',\''+addressLine2+'\',\''+addressLine3+'\',\''+
            city+'\',\''+state+'\',\''+zipCode+'\',\''+data['HOUSEHOLD_ID'][index]+'\',\'Y\')').collect()
    
    
    session.sql('UPDATE '+ sour_table_name+' SET HOUSEHOLDED_IND=\'Y\' where CONTACT_ID=\''+data['CONTACT_ID'][index]+'\'').collect()
    
    session.sql('INSERT INTO '+ dest_table_name+' VALUES ('+ 
                contact_id+',\''+firstName+'\',\'\',\''+lastName+'\',\''+
                addressLine1+'\',\''+addressLine2+'\',\''+addressLine3+'\',\''+
                city+'\',\''+state+'\',\''+zipCode+'\')').collect()
    
    return 0

def noFamilyAssigned(firstName,lastName,addressLine1,addressLine2,addressLine3,city,state,zipCode):
    session = Session.builder.configs(snowflake_conn_prop).create()
    raw = session.table(sour_table_name)
    data= raw.toPandas()
    x=str(random.randrange(10000000, 100000000))
    while x in data['HOUSEHOLD_ID']:
        x=str(random.randrange(10000000, 100000000))
    data['HOUSEHOLD_ID'][len(data)-1]=x
    
    contact_id=str(len(data)+1)
    full_name=firstName+' '+lastName
    session.sql('INSERT INTO '+ sour_table_name+' VALUES ('+ 
            contact_id+',\''+full_name+'\',\''+firstName+'\',\'\',\''+lastName+'\',\''+
            addressLine1+'\',\''+addressLine2+'\',\''+addressLine3+'\',\''+
            city+'\',\''+state+'\',\''+zipCode+'\',\''+x+'\',\'N\')').collect()
    
    session.sql('INSERT INTO '+ dest_table_name+' VALUES ('+ 
                contact_id+',\''+firstName+'\',\'\',\''+lastName+'\',\''+
                addressLine1+'\',\''+addressLine2+'\',\''+addressLine3+'\',\''+
                city+'\',\''+state+'\',\''+zipCode+'\')').collect()
    return 0

def remove(contact_ID):
    session = Session.builder.configs(snowflake_conn_prop).create()
    session.sql('DELETE FROM '+ sour_table_name+' WHERE CONTACT_ID=\''+contact_ID+'\'').collect()
    
    session.sql('DELETE FROM '+ dest_table_name+' WHERE P1_CONTACT_ID=\''+contact_ID+'\'').collect()

def update(contact_ID,firstName,lastName,addressLine1,addressLine2,addressLine3,city,state,zipCode):
    session = Session.builder.configs(snowflake_conn_prop).create()
    full_name=firstName+' '+lastName
    session.sql('UPDATE '+ sour_table_name+' SET FULL_NAME=\''+ 
        full_name+'\',FIRST_NAME=\''+firstName+'\',LAST_NAME=\''+lastName+'\',ADDRESS_LINE_1=\''+
        addressLine1+'\',ADDRESS_LINE_2=\''+addressLine2+'\',ADDRESS_LINE_3=\''+addressLine3+'\',CITY=\''+
        city+'\',STATE=\''+state+'\',ZIP=\''+zipCode+'\' WHERE CONTACT_ID=\''+contact_ID+'\'').collect()
    
    session.sql('UPDATE '+ dest_table_name+' SET P1_FIRSTNAME=\''+firstName+'\',P1_LASTNAME=\''+lastName+'\',P1_ADDRESS_LINE_1=\''+
        addressLine1+'\',P1_ADDRESS_LINE_2=\''+addressLine2+'\',P1_ADDRESS_LINE_3=\''+addressLine3+'\',P1_CITY=\''+
        city+'\',P1_STATE=\''+state+'\',P1_ZIP=\''+zipCode+'\' WHERE P1_CONTACT_ID=\''+contact_ID+'\'').collect()
    
def find(contact_ID):
    session = Session.builder.configs(snowflake_conn_prop).create()
    raw = session.table(dest_table_name)

    return [session.table(sour_table_name).toPandas().iloc[int(contact_ID)-1],
            session.table(dest_table_name).toPandas().iloc[int(contact_ID)-1]]

In [2]:
initialization()

## Registrate a new user, with no existed household relationship

In [3]:
register("Xuran","Cai","W Johnson ST","","Madison WI 53715","Madison","WI","53715")

session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
raw = session.table(sour_table_name)
data=raw.toPandas()
data.tail(1)

[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]
[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]


,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLDED_IND
1001,1002,Xuran Cai,Xuran,,Cai,W Johnson ST,,Madison WI 53715,Madison,WI,53715,40904669,N


In [4]:
session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
raw = session.table(dest_table_name)
data=raw.toPandas()
data.tail(1)

[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]


,P1_CONTACT_ID,P1_FIRSTNAME,P1_MIDDLENAME,P1_LASTNAME,P1_ADDRESS_LINE_1,P1_ADDRESS_LINE_2,P1_ADDRESS_LINE_3,P1_CITY,P1_STATE,P1_ZIP
1001,1002,Xuran,,Cai,W Johnson ST,,Madison WI 53715,Madison,WI,53715


# Registrate an existing user(without household relationship) again, it should now form household with himself/herself

In [5]:
#register("Xuran","Cai","W Johnson ST","","Madison WI 53715","Madison","WI","53715")
register("Claire1","Walls","1031189 Metus St","","Eugene MD 22207","Eugene","MD","22207")
session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
raw = session.table('XURAN_TABLE')
data=raw.toPandas()
#data.tail(3)
data.tail(2)

[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]
[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]


,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLDED_IND
1001,1002,Xuran Cai,Xuran,,Cai,W Johnson ST,,Madison WI 53715,Madison,WI,53715,40904669,N
1002,1003,Claire1 Walls,Claire1,,Walls,1031189 Metus St,,Eugene MD 22207,Eugene,MD,22207,99900001,Y


In [6]:
data.head(1)

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLDED_IND
0,1,Claire Walls,Claire,None,Walls,1031189 Metus St,None,Eugene MD 22207,Eugene,MD,22207,99900001,Y


In [7]:
session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
raw = session.table(dest_table_name)
data=raw.toPandas()
data.tail(3)

[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]


,P1_CONTACT_ID,P1_FIRSTNAME,P1_MIDDLENAME,P1_LASTNAME,P1_ADDRESS_LINE_1,P1_ADDRESS_LINE_2,P1_ADDRESS_LINE_3,P1_CITY,P1_STATE,P1_ZIP
1000,1001,Mary,,Barnett,6991 Vulputate Rd,,Covington KS 61337,Covington,KS,61337
1001,1002,Xuran,,Cai,W Johnson ST,,Madison WI 53715,Madison,WI,53715
1002,1003,Claire1,,Walls,1031189 Metus St,,Eugene MD 22207,Eugene,MD,22207


In [8]:
data.head(1)

,P1_CONTACT_ID,P1_FIRSTNAME,P1_MIDDLENAME,P1_LASTNAME,P1_ADDRESS_LINE_1,P1_ADDRESS_LINE_2,P1_ADDRESS_LINE_3,P1_CITY,P1_STATE,P1_ZIP
0,1,Claire,None,Walls,1031189 Metus St,None,Eugene MD 22207,Eugene,MD,22207


# Update one existing user; its data should be change accordingly

In [9]:
update("1","Claire1","Walls","1031189 Metus St","","Eugene MD 22207","Eugene","MD","22207")
session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
raw = session.table(sour_table_name)
data=raw.toPandas()
data.head(1)

[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]


,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLDED_IND
0,1,Claire1 Walls,Claire1,None,Walls,1031189 Metus St,,Eugene MD 22207,Eugene,MD,22207,99900001,Y


In [10]:
session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
raw = session.table(dest_table_name)
data=raw.toPandas()
data.head(1)

[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]


,P1_CONTACT_ID,P1_FIRSTNAME,P1_MIDDLENAME,P1_LASTNAME,P1_ADDRESS_LINE_1,P1_ADDRESS_LINE_2,P1_ADDRESS_LINE_3,P1_CITY,P1_STATE,P1_ZIP
0,1,Claire1,None,Walls,1031189 Metus St,,Eugene MD 22207,Eugene,MD,22207


# Removing a user; we just remove the account but not existing hoursehold relationship

In [11]:
remove("1")
session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
raw = session.table(sour_table_name)
data=raw.toPandas()
data.head(1)

[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]


,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLDED_IND
0,2,Derek Kane,Derek,None,Kane,1055973 Semper Av,None,Helena OH 24436,Helena,OH,24436,99900002,N


In [12]:
data.tail(1)

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLDED_IND
1001,1003,Claire1 Walls,Claire1,,Walls,1031189 Metus St,,Eugene MD 22207,Eugene,MD,22207,99900001,Y


In [13]:
session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())
raw = session.table(dest_table_name)
data=raw.toPandas()
data.head(1)

[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='TEAM1_XURAN')]


,P1_CONTACT_ID,P1_FIRSTNAME,P1_MIDDLENAME,P1_LASTNAME,P1_ADDRESS_LINE_1,P1_ADDRESS_LINE_2,P1_ADDRESS_LINE_3,P1_CITY,P1_STATE,P1_ZIP
0,2,Derek,None,Kane,1055973 Semper Av,None,Helena OH 24436,Helena,OH,24436


In [14]:
data.tail(1)

,P1_CONTACT_ID,P1_FIRSTNAME,P1_MIDDLENAME,P1_LASTNAME,P1_ADDRESS_LINE_1,P1_ADDRESS_LINE_2,P1_ADDRESS_LINE_3,P1_CITY,P1_STATE,P1_ZIP
1001,1003,Claire1,,Walls,1031189 Metus St,,Eugene MD 22207,Eugene,MD,22207


# Find a user with contact ID

In [15]:
find("1")

[CONTACT_ID                         2
 FULL_NAME                 Derek Kane
 FIRST_NAME                     Derek
 MIDDLE_NAME                     None
 LAST_NAME                       Kane
 ADDRESS_LINE_1     1055973 Semper Av
 ADDRESS_LINE_2                  None
 ADDRESS_LINE_3       Helena OH 24436
 CITY                          Helena
 STATE                             OH
 ZIP                            24436
 HOUSEHOLD_ID                99900002
 HOUSEHOLDED_IND                    N
 Name: 0, dtype: object,
 P1_CONTACT_ID                        2
 P1_FIRSTNAME                     Derek
 P1_MIDDLENAME                     None
 P1_LASTNAME                       Kane
 P1_ADDRESS_LINE_1    1055973 Semper Av
 P1_ADDRESS_LINE_2                 None
 P1_ADDRESS_LINE_3      Helena OH 24436
 P1_CITY                         Helena
 P1_STATE                            OH
 P1_ZIP                           24436
 Name: 0, dtype: object]